In [ ]:
!pip install irsx

In [13]:
!irsx_index --year=2021 --verbose

Getting index file for year: 2021 remote=https://s3.amazonaws.com/irs-form-990/index_2021.csv local=/opt/conda/miniconda3/lib/python3.8/site-packages/irsx/CSV/index_2021.csv
Beginning streaming download of https://s3.amazonaws.com/irs-form-990/index_2021.csv
Total file size: 55.76 MB
Download completed to /opt/conda/miniconda3/lib/python3.8/site-packages/irsx/CSV/index_2021.csv in 0:00:01.670319


In [15]:
!head -3 /opt/conda/miniconda3/lib/python3.8/site-packages/irsx/CSV/index_2021.csv

RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
17606342,EFILE,452772761,201906,1/21/2021 10:02:51 AM,CAMDENS CHARTER SCHOOL NETWORK INC,990,93493065013010,202010659349301301,
17606343,EFILE,237061115,201906,1/21/2021 10:02:51 AM,JACKSON STATE UNIVERSITY DEVELOPMENT FOUNDATION INC,990,93493072000410,202010729349300041,


In [8]:
import pandas as pd

df21 = pd.read_csv("/opt/conda/miniconda3/lib/python3.8/site-packages/irsx/CSV/index_2021.csv",index_col=False)
object_ids = df21["OBJECT_ID"].to_numpy()

/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
object_ids_s = object_ids[:10]
object_ids_s

array([202010659349301301, 202010729349300041, 202010729349300021,
       202010729349300836, 202000669349300135, 202000669349301125,
       202000669349301320, 202000669349301850, 202000669349300905,
       202000669349300700])

In [10]:
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
list_all = []
for e in object_ids_s:
    filing = xml_runner.run_filing(e)
    schedules = filing.list_schedules()
    
    ein = 0
    state = 0
    name = 0
    revenue = 0
    revenueEZ = 0
    
    if "ReturnHeader990x" in schedules:
        header = filing.get_parsed_sked("ReturnHeader990x")
        #print(len(header))
        header_part_i = header[0]["schedule_parts"]["returnheader990x_part_i"]
        #print(header_part_i)
        ein = header_part_i["ein"]
        try:
            state = header_part_i["USAddrss_SttAbbrvtnCd"]
        except KeyError:
            state = XX
        name = header_part_i["BsnssNm_BsnssNmLn1Txt"]
        #print(ein, state, name)
        
    if "IRS990EZ" in schedules:
        irs990ez = filing.get_parsed_sked("IRS990EZ")
        irs990ez_part_i = irs990ez[0]["schedule_parts"]["ez_part_i"]
        #revenue = irs990ez_part_i["TtlRvnAmt"] 
        revenueEZ = irs990ez_part_i["TtlRvnAmt"]  
    
    if "IRS990" in schedules:
        irs990 = filing.get_parsed_sked("IRS990")
        #print(len(irs990))
        irs990_part_i = irs990[0]["schedule_parts"]["part_i"]
        revenue = irs990_part_i["CYTtlRvnAmt"]
        #print(revenue)
    
    revenue = int(revenue) + int(revenueEZ)
        
    list = [e, ein, state, name, revenue]
    list_all.append(list)
    
df_result = pd.DataFrame(list_all, columns =['ObjectID', 'EIN', 'State', 'Name', 'Revenue']) #, 'RevenueEZ'])
df_result.to_csv("hdfs://cluster-jupyter-mw-m/user/hdfs/spark_csv/revenue_2021_100.csv", index = False)
df_result
  

        
    #print(e, ein, state, name, revenue, revenueEZ)

2022-01-14 09:57:35,037 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-01-14 09:57:36,133 WARN shortcircuit.DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.


,ObjectID,EIN,State,Name,Revenue
0,202010659349301301,452772761,NJ,CAMDEN'S CHARTER SCHOOL,5148336
1,202010729349300041,237061115,MS,JACKSON STATE UNIVERSITY,10386900
2,202010729349300021,344427516,OH,TIFFIN UNIVERSITY,64789812
3,202010729349300836,840865247,CO,NETWORK MINISTRIES INC,218029
4,202000669349300135,205647589,SC,FAMILY PROMISE OF BEAUFORT COUNTY,482668
5,202000669349301125,311131742,CA,Pacific Local Masters Swimming Committee,143394
6,202000669349301320,271969337,OR,ST MARYS OUTREACH INC,105375
7,202000669349301850,474824100,TN,PROJECT HEART,155603
8,202000669349300905,930955648,WA,COLUMBIA GORGE INTERPRETIVE CENTER,377646
9,202000669349300700,270454243,CA,CHILDREN OF CONSERVATION INC,132109


In [11]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext(appName="BigDataIRSSpark")
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=BigDataIRSSpark, master=yarn) created by __init__ at /tmp/ipykernel_23243/1525508950.py:3 

In [3]:
#csv_path = "gs://sparkbucket02/revenue_2021_100.csv"
#gs://movie_files_101/u.item


# so wie ich das verstanden habe wird nun auf das lokale File zugegriffen ..
# sprich wenn wir mehrere Worker Nodes haben, müssen alle Worker auch dieses File haben => funktioniert wohl, aber HDFS benutzen wäre wohl cooler
# https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
csv_path = "file:///BIGData/revenue_2021_100.csv"
csv_df = spark.read.csv(csv_path, header=True)
csv_df.head(10)

22/01/14 09:46:49 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 1.0 (TID 4) (cluster-jupyter-mw-w-2.us-east1-c.c.fleet-parity-334009.internal executor 1): java.io.FileNotFoundException: File file:/BIGData/revenue_2021_100.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1

Py4JJavaError: An error occurred while calling o61.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 7) (cluster-jupyter-mw-w-2.us-east1-c.c.fleet-parity-334009.internal executor 1): java.io.FileNotFoundException: File file:/BIGData/revenue_2021_100.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNex

In [12]:
#hdfs_csv_path = "hdfs://spark-jupyter-m/spark_csv/revenue_2021_100.csv"
#hdfs_csv_path = "hdfs:///spark_csv/revenue_2021_100.csv"
#hdfs_csv_path = "spark_csv/revenue_2021_100.csv"
hdfs_csv_path = "hdfs://cluster-jupyter-mw-m/user/hdfs/spark_csv/revenue_2021_100.csv"
hdfs_df = spark.read.csv(hdfs_csv_path, header = True, inferSchema = True)
#hdfs_df.parallelize()
#hdfs_df.head(100)
#hdfs_df.collect()
hdfs_df.printSchema()

hdfs_drop = hdfs_df.drop('ObjectID','EIN','Name','RevenueEZ')
#hdfs_drop.printSchema()

hdfs_rdd = hdfs_drop.rdd
#hdfs_rdd.collect()

from operator import add
reduced_rdd = hdfs_rdd.reduceByKey(add).sortBy(lambda x: x[1], ascending = False)
reduced_rdd.collect()


#sparkTuple = hdfs_rdd.map(lambda letter: (RA(letter), letter))

#rdd.flatMap(lambda x: [(x, x), (x, x)]).collect())

root
 |-- ObjectID: long (nullable = true)
 |-- EIN: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Revenue: integer (nullable = true)



[('OH', 64789812),
 ('MS', 10386900),
 ('NJ', 5148336),
 ('SC', 482668),
 ('WA', 377646),
 ('CA', 275503),
 ('CO', 218029),
 ('TN', 155603),
 ('OR', 105375)]

In [7]:
#from pyspark import SparkContext
#from pyspark.sql import SQLContext
#from pyspark import SparkFiles
#sc = SparkContext(appName="BigDataIRS")
#sc.setLogLevel("ERROR")
#sqlContext = SQLContext(sc)
#df = sqlContext.read.csv("hdfs:///spark_csv/revenue_2021_100.csv", header=True, inferSchema= True)
#df = sqlContext.read.csv("hdfs:///spark_csv/revenue_2021_100.csv", header=True, inferSchema= True)
#df.head()

In [1]:
#from google.cloud import storage
#client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
#bucket = client.get_bucket('sparkbucket02')
# Then do other things...
#blob = bucket.get_blob("revenue_2021_100.csv") #('remote/path/to/file.txt')

#df = pd.read_csv(blob.download_as_string())
#df

In [2]:
#spark = SparkSession \
#    .builder \
#    .appName("Protob Conversion to Parquet") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()\

#df = spark.read.csv('/home/hadoop/observations_temp.csv, header=True)

sudo su - hdfs
hdfs dfsadmin -safemode leave

hdfs dfs -mkdir spark_csv
hdfs dfs -put /spark_csv/revenue_2021_100.csv spark_csv/revenue_2021_100.csv
hdfs dfs -ls spark_csv

https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
https://stackoverflow.com/questions/33055403/how-to-navigate-directories-in-hadoop-hdfs
https://stackoverflow.com/questions/28213116/hadoop-copy-a-local-file-system-folder-to-hdfs
https://stackoverflow.com/questions/61197811/can-i-read-csv-files-from-google-storage-using-spark-in-more-than-one-executor
https://groups.google.com/g/cloud-dataproc-discuss/c/cubkWrjkk2g?pli=1
https://stackoverflow.com/questions/56448009/storing-source-file-in-google-dataproc-hdfs-vs-google-cloud-storagegoogle-bucke

hdfs dfs -put /spark_csv/revenue_2021_100.csv /user/root/spark_csv/revenue_2021_100.csv